<a href="https://colab.research.google.com/github/atick-faisal/MultiViewUNet-Aneurysm/blob/main/src/training/MultiViewUNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Colab Info


In [1]:
from psutil import virtual_memory

gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
    print('Not connected to a GPU')
else:
    print(gpu_info)


ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
    print('Not using a high-RAM runtime')
else:
    print('You are using a high-RAM runtime!')


Sun Jun 18 21:45:24 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   45C    P8    10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# Fix for GDrive


In [2]:
!pip install -U --no-cache-dir gdown --pre > /dev/null


# Mount GDrive


In [3]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


# Download and Extract Dataset


In [4]:
!gdown "19lTs5vzsJ619OeW6fMhwxJACkY-EKrs3"
!unzip -o "AAA_DATASET_r1.zip" > /dev/null


Downloading...
From (uriginal): https://drive.google.com/uc?id=19lTs5vzsJ619OeW6fMhwxJACkY-EKrs3
From (redirected): https://drive.google.com/uc?id=19lTs5vzsJ619OeW6fMhwxJACkY-EKrs3&confirm=t&uuid=8d1f1903-54bb-47ab-aa24-6e359016f998
To: /content/AAA_DATASET_r1.zip
100% 3.07G/3.07G [00:46<00:00, 65.6MB/s]


# Imports


In [5]:
import os
import datetime
import matplotlib
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt

from tqdm import tqdm

matplotlib.use('Agg')
plt.rcParams["font.size"] = 16


# Config


In [6]:
PROBLEM = "Curvature_2_ECAP"

MODEL_NAME = "MultiViewUNet"
DATASET_PATH = "/content/Images/"
TRAIN_DIR = "Train/"
TEST_DIR = "Test/"
INPUT_DIR = PROBLEM.split("_2_")[0]
TARGET_DIR = PROBLEM.split("_2_")[1]
MODEL_PATH = "/content/drive/MyDrive/Research/TAVI/Models/"
PRED_PATH = "/content/drive/MyDrive/Research/TAVI/Predictions/"
IMG_SIZE = 256
BATCH_SIZE = 16
BUFFER_SIZE = 1000
VAL_SPLIT = 0.2
LEARNING_RATE = 0.001
N_EPOCHS = 300
PATIENCE = 30

EXP_NAME = f"{PROBLEM}_{MODEL_NAME}_I{IMG_SIZE}_B{BATCH_SIZE}_LR{LEARNING_RATE}"


# Architecture


In [7]:
class UNet:
    def __init__(
        self,
        img_size: int,
        n_channels: int = 3,
        width: int = 32,
        depth: int = 4,
        kernel_size: int = 3
    ):
        self.img_size = img_size
        self.n_channels = n_channels
        self.width = width
        self.depth = depth
        self.kernel_size = kernel_size

    @staticmethod
    def conv(
        x: tf.Tensor,
        filters: int,
        kernel_size: int
    ) -> tf.Tensor:
        for i in range(2):
            x = tf.keras.layers.Conv2D(
                filters=filters,
                kernel_size=kernel_size,
                strides=1,
                padding="same",
                data_format="channels_last",
                dilation_rate=1,
                groups=1,
                activation=None,
                use_bias=True,
                kernel_initializer="glorot_uniform",
                bias_initializer="zeros",
            )(x)

            x = tf.keras.layers.BatchNormalization()(x)
            x = tf.keras.layers.Activation("relu")(x)

        return x

    @staticmethod
    def deconv(x: tf.Tensor, filters: int) -> tf.Tensor:
        x = tf.keras.layers.Conv2DTranspose(
            filters=filters,
            kernel_size=2,
            strides=2,
            padding="same",
            output_padding=None,
            data_format=None,
            dilation_rate=1,
            activation=None,
            use_bias=True,
            kernel_initializer="glorot_uniform",
            bias_initializer="zeros",
        )(x)

        x = tf.keras.layers.BatchNormalization()(x)
        x = tf.keras.layers.Activation("relu")(x)

        return x

    @staticmethod
    def output(x: tf.Tensor) -> tf.Tensor:
        return tf.keras.layers.Conv2D(3, (1, 1), activation="sigmoid")(x)

    @staticmethod
    def pool(x: tf.Tensor, pool_size: int = 2) -> tf.Tensor:
        return tf.keras.layers.MaxPool2D(pool_size)(x)

    @staticmethod
    def dropout(x: tf.Tensor, amount: float = 0.5) -> tf.Tensor:
        return tf.keras.layers.Dropout(amount)(x)

    def __call__(self) -> tf.keras.Model:
        inputs = tf.keras.layers.Input(
            shape=(self.img_size, self.img_size, self.n_channels)
        )

        # scaled = tf.keras.layers.Rescaling(1./255.0, offset=0)(inputs)

        # ------------------ Downsampling ---------------------
        downsample_layers = []
        downsample_layers.append(
            self.conv(
                x=inputs,
                filters=self.width,
                kernel_size=self.kernel_size
            )
        )
        for i in range(1, self.depth):
            dropout_amount = 0.2 if i == 1 else 0.5
            filters = int((2 ** i) * self.width)
            downsample_layers.append(
                self.dropout(
                    self.pool(
                        self.conv(
                            x=downsample_layers[i - 1],
                            filters=filters,
                            kernel_size=self.kernel_size
                        )
                    ),
                    amount=dropout_amount
                )
            )

        # ------------------- Features --------------------
        n_features = int((2 ** self.depth) * self.width)
        self.features = self.pool(
            self.conv(
                x=downsample_layers[-1],
                filters=n_features,
                kernel_size=self.kernel_size
            )
        )

        # ------------------- Upsampling --------------------
        upsample_layers = []
        upsample_layers.append(self.features)
        for i in range(1, self.depth + 1):
            filters = int((2 ** (self.depth - i)) * self.width)
            upsample_layers.append(
                self.conv(
                    x=self.dropout(
                        tf.keras.layers.concatenate([
                            downsample_layers[self.depth - i],
                            self.deconv(
                                x=upsample_layers[i - 1],
                                filters=filters
                            )
                        ])
                    ),
                    filters=filters,
                    kernel_size=self.kernel_size
                )
            )

        # ---------------------- Output -----------------------
        outputs = self.output(upsample_layers[-1])

        return tf.keras.Model(inputs, outputs)


In [8]:
# ... MultiResUNet
from keras.layers import Input, Conv2D, MaxPooling2D, Conv2DTranspose, concatenate, BatchNormalization, Activation, add
from keras.models import Model, model_from_json
from keras.optimizers import Adam
# from keras.layers.advanced_activations import ELU, LeakyReLU
from keras.utils.vis_utils import plot_model


def conv2d_bn(x, filters, num_row, num_col, padding='same', strides=(1, 1), activation='relu', name=None):
    '''
    2D Convolutional layers

    Arguments:
        x {keras layer} -- input layer
        filters {int} -- number of filters
        num_row {int} -- number of rows in filters
        num_col {int} -- number of columns in filters

    Keyword Arguments:
        padding {str} -- mode of padding (default: {'same'})
        strides {tuple} -- stride of convolution operation (default: {(1, 1)})
        activation {str} -- activation function (default: {'relu'})
        name {str} -- name of the layer (default: {None})

    Returns:
        [keras layer] -- [output layer]
    '''

    x = Conv2D(filters, (num_row, num_col), strides=strides,
               padding=padding, use_bias=False)(x)
    x = BatchNormalization(axis=3, scale=False)(x)

    if(activation == None):
        return x

    x = Activation(activation, name=name)(x)

    return x


def trans_conv2d_bn(x, filters, num_row, num_col, padding='same', strides=(2, 2), name=None):
    '''
    2D Transposed Convolutional layers

    Arguments:
        x {keras layer} -- input layer
        filters {int} -- number of filters
        num_row {int} -- number of rows in filters
        num_col {int} -- number of columns in filters

    Keyword Arguments:
        padding {str} -- mode of padding (default: {'same'})
        strides {tuple} -- stride of convolution operation (default: {(2, 2)})
        name {str} -- name of the layer (default: {None})

    Returns:
        [keras layer] -- [output layer]
    '''

    x = Conv2DTranspose(filters, (num_row, num_col),
                        strides=strides, padding=padding)(x)
    x = BatchNormalization(axis=3, scale=False)(x)

    return x


def MultiResBlock(U, inp, alpha=1.67):
    '''
    MultiRes Block

    Arguments:
        U {int} -- Number of filters in a corrsponding UNet stage
        inp {keras layer} -- input layer

    Returns:
        [keras layer] -- [output layer]
    '''

    W = alpha * U

    shortcut = inp

    shortcut = conv2d_bn(shortcut, int(W*0.167) + int(W*0.333) +
                         int(W*0.5), 1, 1, activation=None, padding='same')

    conv3x3 = conv2d_bn(inp, int(W*0.167), 3, 3,
                        activation='relu', padding='same')

    conv5x5 = conv2d_bn(conv3x3, int(W*0.333), 3, 3,
                        activation='relu', padding='same')

    conv7x7 = conv2d_bn(conv5x5, int(W*0.5), 3, 3,
                        activation='relu', padding='same')

    out = concatenate([conv3x3, conv5x5, conv7x7], axis=3)
    out = BatchNormalization(axis=3)(out)

    out = add([shortcut, out])
    out = Activation('relu')(out)
    out = BatchNormalization(axis=3)(out)

    return out


def ResPath(filters, length, inp):
    '''
    ResPath

    Arguments:
        filters {int} -- [description]
        length {int} -- length of ResPath
        inp {keras layer} -- input layer

    Returns:
        [keras layer] -- [output layer]
    '''

    shortcut = inp
    shortcut = conv2d_bn(shortcut, filters, 1, 1,
                         activation=None, padding='same')

    out = conv2d_bn(inp, filters, 3, 3, activation='relu', padding='same')

    out = add([shortcut, out])
    out = Activation('relu')(out)
    out = BatchNormalization(axis=3)(out)

    for i in range(length-1):

        shortcut = out
        shortcut = conv2d_bn(shortcut, filters, 1, 1,
                             activation=None, padding='same')

        out = conv2d_bn(out, filters, 3, 3, activation='relu', padding='same')

        out = add([shortcut, out])
        out = Activation('relu')(out)
        out = BatchNormalization(axis=3)(out)

    return out


def MultiResUNet(height, width, n_channels):
    '''
    MultiResUNet

    Arguments:
        height {int} -- height of image
        width {int} -- width of image
        n_channels {int} -- number of channels in image

    Returns:
        [keras model] -- MultiResUNet model
    '''

    inputs = Input((height, width, n_channels))

    mresblock1 = MultiResBlock(32, inputs)
    pool1 = MaxPooling2D(pool_size=(2, 2))(mresblock1)
    mresblock1 = ResPath(32, 4, mresblock1)

    mresblock2 = MultiResBlock(32*2, pool1)
    pool2 = MaxPooling2D(pool_size=(2, 2))(mresblock2)
    mresblock2 = ResPath(32*2, 3, mresblock2)

    mresblock3 = MultiResBlock(32*4, pool2)
    pool3 = MaxPooling2D(pool_size=(2, 2))(mresblock3)
    mresblock3 = ResPath(32*4, 2, mresblock3)

    mresblock4 = MultiResBlock(32*8, pool3)
    pool4 = MaxPooling2D(pool_size=(2, 2))(mresblock4)
    mresblock4 = ResPath(32*8, 1, mresblock4)

    mresblock5 = MultiResBlock(32*16, pool4)

    up6 = concatenate([Conv2DTranspose(
        32*8, (2, 2), strides=(2, 2), padding='same')(mresblock5), mresblock4], axis=3)
    mresblock6 = MultiResBlock(32*8, up6)

    up7 = concatenate([Conv2DTranspose(
        32*4, (2, 2), strides=(2, 2), padding='same')(mresblock6), mresblock3], axis=3)
    mresblock7 = MultiResBlock(32*4, up7)

    up8 = concatenate([Conv2DTranspose(
        32*2, (2, 2), strides=(2, 2), padding='same')(mresblock7), mresblock2], axis=3)
    mresblock8 = MultiResBlock(32*2, up8)

    up9 = concatenate([Conv2DTranspose(32, (2, 2), strides=(
        2, 2), padding='same')(mresblock8), mresblock1], axis=3)
    mresblock9 = MultiResBlock(32, up9)

    conv10 = conv2d_bn(mresblock9, 3, 1, 1, activation='sigmoid')

    model = Model(inputs=[inputs], outputs=[conv10])

    return model


# def main():

#     # Define the model

#     model = MultiResUnet(128, 128,3)
#     print(model.summary())


# if __name__ == '__main__':
#     main()


In [9]:
# # ... MultiResUNet3D
# from keras.layers import Input, Conv3D, MaxPooling3D, Conv3DTranspose, concatenate, BatchNormalization, Activation, add
# from keras.models import Model, model_from_json
# from keras.optimizers import Adam
# # from keras.layers.advanced_activations import ELU, LeakyReLU
# from keras.utils.vis_utils import plot_model


# def conv3d_bn(x, filters, num_row, num_col, num_z, padding='same', strides=(1, 1, 1), activation='relu', name=None):
#     '''
#     3D Convolutional layers

#     Arguments:
#         x {keras layer} -- input layer
#         filters {int} -- number of filters
#         num_row {int} -- number of rows in filters
#         num_col {int} -- number of columns in filters
#         num_z {int} -- length along z axis in filters

#     Keyword Arguments:
#         padding {str} -- mode of padding (default: {'same'})
#         strides {tuple} -- stride of convolution operation (default: {(1, 1, 1)})
#         activation {str} -- activation function (default: {'relu'})
#         name {str} -- name of the layer (default: {None})

#     Returns:
#         [keras layer] -- [output layer]
#     '''

#     x = Conv3D(filters, (num_row, num_col, num_z),
#                strides=strides, padding=padding, use_bias=False)(x)
#     x = BatchNormalization(axis=4, scale=False)(x)

#     if(activation == None):
#         return x

#     x = Activation(activation, name=name)(x)
#     return x


# def trans_conv3d_bn(x, filters, num_row, num_col, num_z, padding='same', strides=(2, 2, 2), name=None):
#     '''
#     2D Transposed Convolutional layers

#     Arguments:
#         x {keras layer} -- input layer
#         filters {int} -- number of filters
#         num_row {int} -- number of rows in filters
#         num_col {int} -- number of columns in filters
#         num_z {int} -- length along z axis in filters

#     Keyword Arguments:
#         padding {str} -- mode of padding (default: {'same'})
#         strides {tuple} -- stride of convolution operation (default: {(2, 2, 2)})
#         name {str} -- name of the layer (default: {None})

#     Returns:
#         [keras layer] -- [output layer]
#     '''

#     x = Conv3DTranspose(filters, (num_row, num_col, num_z),
#                         strides=strides, padding=padding)(x)
#     x = BatchNormalization(axis=4, scale=False)(x)

#     return x


# def MultiResBlock(U, inp, alpha=1.67):
#     '''
#     MultiRes Block

#     Arguments:
#         U {int} -- Number of filters in a corrsponding UNet stage
#         inp {keras layer} -- input layer

#     Returns:
#         [keras layer] -- [output layer]
#     '''

#     W = alpha * U

#     shortcut = inp

#     shortcut = conv3d_bn(shortcut, int(W*0.167) + int(W*0.333) +
#                          int(W*0.5), 1, 1, 1, activation=None, padding='same')

#     conv3x3 = conv3d_bn(inp, int(W*0.167), 3, 3, 3,
#                         activation='relu', padding='same')

#     conv5x5 = conv3d_bn(conv3x3, int(W*0.333), 3, 3, 3,
#                         activation='relu', padding='same')

#     conv7x7 = conv3d_bn(conv5x5, int(W*0.5), 3, 3, 3,
#                         activation='relu', padding='same')

#     out = concatenate([conv3x3, conv5x5, conv7x7], axis=4)
#     out = BatchNormalization(axis=4)(out)

#     out = add([shortcut, out])
#     out = Activation('relu')(out)
#     out = BatchNormalization(axis=4)(out)

#     return out


# def ResPath(filters, length, inp):
#     '''
#     ResPath

#     Arguments:
#         filters {int} -- [description]
#         length {int} -- length of ResPath
#         inp {keras layer} -- input layer

#     Returns:
#         [keras layer] -- [output layer]
#     '''

#     shortcut = inp
#     shortcut = conv3d_bn(shortcut, filters, 1, 1, 1,
#                          activation=None, padding='same')

#     out = conv3d_bn(inp, filters, 3, 3, 3, activation='relu', padding='same')

#     out = add([shortcut, out])
#     out = Activation('relu')(out)
#     out = BatchNormalization(axis=4)(out)

#     for i in range(length-1):

#         shortcut = out
#         shortcut = conv3d_bn(shortcut, filters, 1, 1, 1,
#                              activation=None, padding='same')

#         out = conv3d_bn(out, filters, 3, 3, 3,
#                         activation='relu', padding='same')

#         out = add([shortcut, out])
#         out = Activation('relu')(out)
#         out = BatchNormalization(axis=4)(out)

#     return out


# def MultiResUnet3D(height, width, z, n_channels):
#     '''
#     MultiResUNet3D

#     Arguments:
#         height {int} -- height of image
#         width {int} -- width of image
#         z {int} -- length along z axis
#         n_channels {int} -- number of channels in image

#     Returns:
#         [keras model] -- MultiResUNet3D model
#     '''

#     inputs = Input((height, width, z, n_channels))

#     mresblock1 = MultiResBlock(32, inputs)
#     pool1 = MaxPooling3D(pool_size=(2, 2, 2))(mresblock1)
#     mresblock1 = ResPath(32, 4, mresblock1)

#     mresblock2 = MultiResBlock(32*2, pool1)
#     pool2 = MaxPooling3D(pool_size=(2, 2, 2))(mresblock2)
#     mresblock2 = ResPath(32*2, 3, mresblock2)

#     mresblock3 = MultiResBlock(32*4, pool2)
#     pool3 = MaxPooling3D(pool_size=(2, 2, 2))(mresblock3)
#     mresblock3 = ResPath(32*4, 2, mresblock3)

#     mresblock4 = MultiResBlock(32*8, pool3)
#     pool4 = MaxPooling3D(pool_size=(2, 2, 2))(mresblock4)
#     mresblock4 = ResPath(32*8, 1, mresblock4)

#     mresblock5 = MultiResBlock(32*16, pool4)

#     up6 = concatenate([Conv3DTranspose(
#         32*8, (2, 2, 2), strides=(2, 2, 2), padding='same')(mresblock5), mresblock4], axis=4)
#     mresblock6 = MultiResBlock(32*8, up6)

#     up7 = concatenate([Conv3DTranspose(
#         32*4, (2, 2, 2), strides=(2, 2, 2), padding='same')(mresblock6), mresblock3], axis=4)
#     mresblock7 = MultiResBlock(32*4, up7)

#     up8 = concatenate([Conv3DTranspose(
#         32*2, (2, 2, 2), strides=(2, 2, 2), padding='same')(mresblock7), mresblock2], axis=4)
#     mresblock8 = MultiResBlock(32*2, up8)

#     up9 = concatenate([Conv3DTranspose(32, (2, 2, 2), strides=(
#         2, 2, 2), padding='same')(mresblock8), mresblock1], axis=4)
#     mresblock9 = MultiResBlock(32, up9)

#     conv10 = conv3d_bn(mresblock9, 3, 1, 1, 1, activation='sigmoid')

#     model = Model(inputs=[inputs], outputs=[conv10])

#     return model


# # def main():

# #     # Define the model

# #     model = MultiResUnet3D(80, 80, 48, 4)
# #     print(model.summary())


# # if __name__ == '__main__':
# #     main()


# Loss Functions / Metrics


In [10]:
@tf.function
def attention_mse(y_true, y_pred):
    _y_true = y_true[y_true != 1.0]
    _y_pred = y_pred[y_true != 1.0]
    squared_difference = tf.square(_y_true - _y_pred)
    return tf.reduce_mean(squared_difference, axis=-1)


@tf.function
def attention_mae(y_true, y_pred):
    _y_true = y_true[y_true != 1.0]
    _y_pred = y_pred[y_true != 1.0]
    squared_difference = tf.abs(_y_true - _y_pred)
    return tf.reduce_mean(squared_difference, axis=-1)


# DataLoader


In [11]:
def load_data_from_dir(path: str) -> tf.data.Dataset:
    return tf.keras.utils.image_dataset_from_directory(
        directory=path,
        labels=None,
        color_mode='rgb',
        batch_size=BATCH_SIZE,
        image_size=(IMG_SIZE, IMG_SIZE),
        shuffle=False,
        seed=42,
        interpolation='bilinear',
        follow_links=False,
        crop_to_aspect_ratio=False
    )


# Load Dataset


In [12]:
trainX = load_data_from_dir(os.path.join(DATASET_PATH, TRAIN_DIR, INPUT_DIR))
trainY = load_data_from_dir(os.path.join(DATASET_PATH, TRAIN_DIR, TARGET_DIR))
testX = load_data_from_dir(os.path.join(DATASET_PATH, TEST_DIR, INPUT_DIR))
testY = load_data_from_dir(os.path.join(DATASET_PATH, TEST_DIR, TARGET_DIR))

train_ds = tf.data.Dataset.zip((trainX, trainY))
test_ds = tf.data.Dataset.zip((testX, testY))

print(train_ds.element_spec)
print(test_ds.element_spec)


Found 8208 files belonging to 1 classes.
Found 8208 files belonging to 1 classes.
Found 312 files belonging to 1 classes.
Found 312 files belonging to 1 classes.
(TensorSpec(shape=(None, 256, 256, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None, 256, 256, 3), dtype=tf.float32, name=None))
(TensorSpec(shape=(None, 256, 256, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None, 256, 256, 3), dtype=tf.float32, name=None))


# Normalization


In [13]:
normalization_layer = tf.keras.layers.Rescaling(1./255)
train_ds = train_ds.map(lambda x, y: (
    normalization_layer(x), normalization_layer(y)))
test_ds = test_ds.map(lambda x, y: (
    normalization_layer(x), normalization_layer(y)))


# Augmentation


In [14]:
class Augment(tf.keras.layers.Layer):
    def __init__(self, seed=42):
        super().__init__()
        # both use the same seed, so they'll make the same random changes.
        self.augment_inputs = tf.keras.layers.RandomZoom(
            (-0.1, -0.7), seed=seed)
        self.augment_labels = tf.keras.layers.RandomZoom(
            (-0.1, -0.7), seed=seed)

    def call(self, inputs, labels):
        inputs = self.augment_inputs(inputs)
        labels = self.augment_labels(labels)
        return inputs, labels


# Optimization


In [15]:
AUTOTUNE = tf.data.AUTOTUNE

train_batches = (
    train_ds
    .cache()
    .shuffle(BUFFER_SIZE)
    .prefetch(buffer_size=AUTOTUNE)
)

test_batches = (
    test_ds
    .cache()
    .prefetch(buffer_size=AUTOTUNE)
)


# Training Config


In [16]:
model_path = os.path.join(MODEL_PATH, EXP_NAME)

callbacks = [
    tf.keras.callbacks.EarlyStopping(
        monitor="val_loss",
        patience=PATIENCE,
        restore_best_weights=True
    ),
    tf.keras.callbacks.ModelCheckpoint(
        model_path,
        monitor='val_loss',
        verbose=1,
        save_best_only=True,
        save_weights_only=True
    )
]

optimizer = tf.keras.optimizers.Adam(
    learning_rate=LEARNING_RATE
)

model = UNet(IMG_SIZE)()
# model = MultiResUNet(IMG_SIZE, IMG_SIZE, 3)

model.compile(
    loss=attention_mse,
    optimizer=optimizer,
    metrics=[attention_mae]
)

try:
    model.load_weights(model_path)
except:
    print("Checkpoint not found")
    pass


# Training


In [17]:
history = model.fit(
    train_batches,
    validation_data=test_batches,
    epochs=N_EPOCHS,
    batch_size=BATCH_SIZE,
    callbacks=callbacks,
    verbose=1
)


Epoch 1/300
513/513 [==============================] - ETA: 0s - loss: 0.0466 - attention_mae: 0.1527
Epoch 1: val_loss improved from inf to 0.04886, saving model to /content/drive/MyDrive/Research/TAVI/Models/Curvature_2_ECAP_MultiViewUNet_I256_B16_LR0.001
513/513 [==============================] - 604s 815ms/step - loss: 0.0466 - attention_mae: 0.1527 - val_loss: 0.0489 - val_attention_mae: 0.1642
Epoch 2/300
513/513 [==============================] - ETA: 0s - loss: 0.0409 - attention_mae: 0.1399
Epoch 2: val_loss did not improve from 0.04886
513/513 [==============================] - 413s 804ms/step - loss: 0.0409 - attention_mae: 0.1399 - val_loss: 0.0548 - val_attention_mae: 0.1621
Epoch 3/300
513/513 [==============================] - ETA: 0s - loss: 0.0374 - attention_mae: 0.1309
Epoch 3: val_loss improved from 0.04886 to 0.03614, saving model to /content/drive/MyDrive/Research/TAVI/Models/Curvature_2_ECAP_MultiViewUNet_I256_B16_LR0.001
513/513 [==============================] 

# Save Model


In [18]:
timestamp = datetime.datetime.now().strftime('%b-%d-%I:%M%p')
if not os.path.exists(model_path):
    os.makedirs(model_path)

model.save(os.path.join(model_path, timestamp))


In [19]:
os.path.join(model_path, timestamp)

'/content/drive/MyDrive/Research/TAVI/Models/Curvature_2_ECAP_MultiViewUNet_I256_B16_LR0.001/Jun-19-03:01AM'

# Save Predictions


In [20]:
test_ds_unbatched = test_batches.unbatch()

pred_path = os.path.join(PRED_PATH, EXP_NAME, timestamp)
if not os.path.exists(pred_path):
    os.makedirs(pred_path)

metrics = pd.DataFrame()

for idx, (input, target) in enumerate(tqdm(test_ds_unbatched)):

    target = tf.squeeze(target).numpy()
    prediction = tf.squeeze(
        model.predict(
            tf.expand_dims(input, axis=0)
        )
    )

    channel_sum = tf.expand_dims(tf.reduce_sum(target, axis=-1), axis=-1)
    white_mask = tf.reduce_all(tf.equal(channel_sum, 3.0), axis=-1)
    expanded_mask = tf.expand_dims(white_mask, axis=-1)
    expanded_mask = tf.tile(expanded_mask, [1, 1, 3])
    prediction = tf.where(expanded_mask, tf.ones_like(prediction), prediction)

    plt.figure(figsize=(7, 7))
    plt.imshow(target)
    plt.axis("off")
    plt.tight_layout()
    plt.savefig(os.path.join(pred_path, f"{idx}_T.png"))
    plt.close()

    plt.figure(figsize=(7, 7))
    plt.imshow(prediction)
    plt.axis("off")
    plt.tight_layout()
    plt.savefig(os.path.join(pred_path, f"{idx}_P.png"))
    plt.close()


0it [00:00, ?it/s]

1/1 [==============================] - 2s 2s/step


1it [00:03,  3.56s/it]

1/1 [==============================] - 0s 22ms/step


2it [00:04,  1.73s/it]

...

310it [02:20,  2.24it/s]

1/1 [==============================] - 0s 22ms/step


311it [02:21,  2.30it/s]

1/1 [==============================] - 0s 23ms/step


312it [02:21,  2.21it/s]


# Loss Curve


In [21]:
try:
    loss = history.history['loss']
    val_loss = history.history['val_loss']

    plt.figure(figsize=(8, 6))
    plt.plot(loss, label='Training Loss')
    plt.plot(val_loss, label='Validation Loss')
    plt.legend(loc='upper right')
    plt.ylabel('Attention MAE')
    plt.title('Training and Validation Loss')
    plt.xlabel('Epoch')
    plt.tight_layout()
    plt.savefig(os.path.join(model_path, timestamp + ".png"))
    plt.close()
except:
    print("Model did not finish training")


# Metrics


In [22]:
model.evaluate(test_batches)


20/20 [==============================] - 3s 151ms/step - loss: 0.0341 - attention_mae: 0.1265


[0.034087639302015305, 0.12651604413986206]